In [1]:
from itertools import product
import numpy as np
import numpy.random as rn
import pandas as pd
import math
from datetime import timedelta
from datetime import datetime 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
result_df = pd.read_excel('./sample data.xlsx')

In [4]:
user_trajectories = []

for user_id, user_data in result_df.groupby('user_id'):
    trajectory = []

    for _, row in user_data.iterrows():
        state_action = [row['IS'], row['ES'],row['COM'],row['initiate'],row['self_reply'],row['reply_to_others']]
        trajectory.append(state_action)
    user_trajectories.append(np.array(trajectory))
user_trajectories = [trajectory.astype(int) for trajectory in user_trajectories]

In [5]:
s_dict={0: [0, 0, 0,0,0,0], 1: [0, 0, 0,0,0,1], 2: [0, 0,0,0,1, 0], 3: [0,0,0,1, 0, 0],4:[0,0,1,0,0,1],5:[0,0,1,0,1,0],6:[0,0,1,1,0,0],7:[0,1,0,0,0,1],8:[0,1,0,0,1,0],9:[0,1,0,1,0,0],10:[1,0,0,0,0,1],11:[1,0,0,0,1,0],12:[1,0,0,1,0,0],13:[0,0,1,0,0,0],14:[0,1,0,0,0,0],15:[1,0,0,0,0,0],16:[0,1,1,0,0,1],17:[1,0,1,0,0,1],18:[1,1,0,0,0,1],19:[1,1,1,0,0,1],20:[0,1,1,0,1,0],21:[1,0,1,0,1,0],22:[1,1,0,0,1,0],23:[1,1,1,0,1,0],24:[0,1,1,1,0,0],25:[1,0,1,1,0,0],26:[1,1,0,1,0,0],27:[1,1,1,1,0,0],28:[0,1,1,0,0,0],29:[1,0,1,0,0,0],30:[1,1,0,0,0,0],31:[1,1,1,0,0,0]}
a_dict = a_dict={0: [0, 0, 0], 1: [0, 0, 1], 2: [0, 1, 0], 3: [1, 0, 0]}# actions dictionary
feature_matrix=[] # feature matrix preparation
for el in s_dict:
    array=s_dict[el]
    int_array = [ int(s) for s in array ]
    feature_matrix.append(int_array)
feature_matrix = np.asarray(feature_matrix)   
c_dict={}
for i in np.arange(len(s_dict)):
    c_dict[i]=0

n_actions = len(a_dict)
n_states = len(s_dict)

In [6]:
def value(policy, n_states, transition_probabilities, reward, discount,threshold=1e-2):
    v = np.zeros(n_states)
    diff = float("inf")
    while diff > threshold:
        diff = 0
        for s in range(n_states):
            vs = v[s]
            a = policy[s]
            v[s] = sum(transition_probabilities[s, a, k] *
                       (reward[k] + discount * v[k])
                       for k in range(n_states))
            diff = max(diff, abs(vs - v[s]))

    return v
    
def optimal_value(n_states, n_actions, transition_probabilities, reward,discount, threshold=1e-2):
    v = np.zeros(n_states)
    diff = float("inf")
    while diff > threshold:
        diff = 0
        for s in range(n_states):
            max_v = float("-inf")
            for a in range(n_actions):
                tp = transition_probabilities[s, a, :]
                max_v = max(max_v, np.dot(tp, reward + discount*v))
            new_diff = abs(v[s] - max_v)
            if new_diff > diff:
                diff = new_diff
            v[s] = max_v
    return v

def find_policy(n_states, n_actions, transition_probabilities, reward, discount,threshold=1e-2, v=None, stochastic=True):
    if v is None:
        v = optimal_value(n_states, n_actions, transition_probabilities,reward,
                          discount, threshold)
    if stochastic:
        Q = np.zeros((n_states, n_actions))
        for i in range(n_states):
            for j in range(n_actions):
                p = transition_probabilities[i, j, :]
                Q[i, j] = p.dot(reward + discount*v)
        Q -= Q.max(axis=1).reshape((n_states, 1))  # For numerical stability.
        Q = np.exp(Q)/np.exp(Q).sum(axis=1).reshape((n_states, 1))
        return Q
    def _policy(s):
        return max(range(n_actions),
                   key=lambda a: sum(transition_probabilities[s, a, k] *
                                     (reward[k] + discount * v[k])
                                     for k in range(n_states)))
    policy = np.array([_policy(s) for s in range(n_states)])
    return policy

In [7]:
def irl(feature_matrix, n_actions, discount, transition_probability,
        trajectories, epochs, learning_rate):
    n_states, d_states = feature_matrix.shape
    # Initialise weights.
    alpha = rn.uniform(size=(d_states,))
    # Calculate the feature expectations \tilde{phi}.
    feature_expectations = find_feature_expectations(feature_matrix, trajectories)
    # Gradient descent on alpha.
    for i in range(epochs):
        # print("i: {}".format(i))
        r = feature_matrix.dot(alpha)
        expected_svf = find_expected_svf(n_states, r, n_actions, discount, transition_probability, trajectories)
        grad = feature_expectations - feature_matrix.T.dot(expected_svf)
        alpha += learning_rate * grad
    return feature_matrix.dot(alpha).reshape((n_states,))

In [8]:
def find_svf(n_states, trajectories):
    svf = np.zeros(n_states)
    for trajectory in trajectories:
        #for state, _, _ in trajectory:
        for state, _ in trajectory:
            svf[state] += 1
    svf /= trajectories.shape[0]
    return svf

def find_feature_expectations(feature_matrix, trajectories):
    feature_expectations = np.zeros(feature_matrix.shape[1])
    total_states = 0
    
    for trajectory in trajectories:
        for state in trajectory: 
            # Map the state to an integer index and add the corresponding row to feature_expectations.
            state_index = np.argmax(state) 
            feature_expectations += feature_matrix[state_index]

            total_states += 1

    # Normalize by the total number of states.
    feature_expectations /= total_states

    return feature_expectations

# from itertools import product

def find_expected_svf(n_states, r, n_actions, discount, transition_probability, trajectories):
    n_trajectories = len(trajectories)
    trajectory_length = len(trajectories[0]) 
    policy = find_policy(n_states, n_actions, transition_probability, r, discount)
    start_state_count = np.zeros(n_states)

    for trajectory in trajectories:
        start_state_count[trajectory[0]] += 1

    p_start_state = start_state_count / n_trajectories

    expected_svf = np.tile(p_start_state, (trajectory_length, 1)).T
    for t in range(1, trajectory_length):
        expected_svf[:, t] = 0
        for i, j, k in product(range(n_states), range(n_actions), range(n_states)):
            expected_svf[k, t] += (expected_svf[i, t-1] *
                                  policy[i, j] *
                                  transition_probability[i, j, k])

    return expected_svf.sum(axis=1)

def softmax(x1, x2):
    max_x = max(x1, x2)
    min_x = min(x1, x2)
    return max_x + np.log(1 + np.exp(min_x - max_x))

def find_policy1(n_states, r, n_actions, discount,
                           transition_probability):
    V = np.nan_to_num(np.ones((n_states, 1)) * float("-inf"))

    diff = np.ones((n_states,))
    while (diff > 1e-4).all():  # Iterate until convergence.
        new_V = r.copy()
        for j in range(n_actions):
            for i in range(n_states):
                new_V[i] = softmax(new_V[i], r[i] + discount*
                    np.sum(transition_probability[i, j, k] * V[k]
                           for k in range(n_states)))

        # # This seems to diverge, so we z-score it (engineering hack).
        new_V = (new_V - new_V.mean())/new_V.std()

        diff = abs(V - new_V)
        V = new_V

    
    Q = np.zeros((n_states, n_actions))
    for i in range(n_states):
        for j in range(n_actions):
            p = np.array([transition_probability[i, j, k]
                          for k in range(n_states)])
            Q[i, j] = p.dot(r + discount*V)

    # Softmax by row to interpret these values as probabilities.
    Q -= Q.max(axis=1).reshape((n_states, 1))  # For numerical stability.
    Q = np.exp(Q)/np.exp(Q).sum(axis=1).reshape((n_states, 1))
    return Q

def expected_value_difference(n_states, n_actions, transition_probability,reward, discount, p_start_state, optimal_value, true_reward):
    policy = value_iteration.find_policy(n_states, n_actions,transition_probability, reward, discount)
    value = value_iteration.value(policy.argmax(axis=1), n_states,transition_probability, true_reward, discount)
    evd = optimal_value.dot(p_start_state) - value.dot(p_start_state)
    return evd

def compute_tp(state_sequences, n_states, n_actions):
    tp = np.zeros([n_states, n_actions, n_states])

    for user_trajectory in state_sequences:
        for pair in np.arange(len(user_trajectory) - 1):
            s = user_trajectory[pair][0]
            a = user_trajectory[pair][1]
            ns = user_trajectory[pair + 1][0]
            tp[s, a, ns] += 1

    for c in np.arange(len(tp)):
        for a in np.arange(len(tp[c])):
            total_transitions = tp[c, a].sum()
            if total_transitions > 0:
                tp[c, a] /= total_transitions

    return tp

In [9]:
# IRL parameters
discount = 0.9
epochs = 1000
learning_rate = 0.01
l1 = l2 = 0
tp= compute_tp(user_trajectories,n_states,n_actions) # compute transition probabilities 

In [10]:
all_results3 = []
for users in user_trajectories:
    r = irl(feature_matrix, n_actions, discount, tp, user_trajectories, epochs,learning_rate) 
    all_results3.append(r)
all_results = pd.DataFrame(all_results3)

In [11]:
all_results.to_excel('./reward.xlsx')